In [1]:
def draw_hard(dst=151, v=1.0, x=6):
    filename = "trudne{}".format(x)
    img = io.imread('../res/'+filename+'.jpg')
    image = adjust_gamma(img, v)
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blur = cv.medianBlur(gray, 5)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv.filter2D(blur, -1, sharpen_kernel)

    #thresh = cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    thresh = cv.threshold(sharpen,dst,255,cv.THRESH_BINARY_INV)[1]
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
    close = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
    #figure(figsize=(20,20))
    plt.imshow(close)
    cnts = cv.findContours(close, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    fields = []

    for c in cnts:
        area = cv.contourArea(c)
        fields.append(area)
    
    index = fields.index(max(fields))
    del cnts[index]
    del fields[index]
    biggest = max(fields)
    average = sum(fields)/len(fields)
        
    fields2 = fields.copy()
    dlugosc = len(fields2)
    i = 0
    while(i < dlugosc):
        if fields2[i] < max(fields2)//2:
            del fields2[i]
            del cnts[i]
            dlugosc-=1
        else:
            i+=1

    zooms = []
    coords = []
    for i, j in zip(cnts, fields):
        if j <= biggest:
            x,y,w,h = cv.boundingRect(i)
            zoom = image[y:y+h, x:x+h]
            coords.append([y+h//2, x+h//2])
            zooms.append(zoom)
    #print(len(zooms))
    #for i in range(0, len(zooms)):
    #    figure(figsize=(6,6))
    #    plt.imshow(zooms[i])
    return zooms, coords
#interact(draw_hard, dst=(0.0,255.0,1.0), v=(0.5,4.0,0.01), x=(0,9,1))

In [2]:
hard3_zoom, hard3_contours = draw_hard(115, 1.5, 3)
hard4_zoom, hard4_contours = draw_hard(222, 2.38, 4)
hard5_1_zoom, hard5_1_contours = draw_hard(52, 0.93, 5)
hard5_2_zoom, hard5_2_contours = draw_hard(52, 1.4, 5)
hard5_zoom = hard5_1_zoom
hard5_contours = hard5_1_contours
for i, j in zip(hard5_2_zoom, hard5_2_contours):
    hard5_zoom.append(i)
    hard5_contours.append(j)
hard6_zoom, hard6_contours = draw_hard(165, 3.8, 6)

zooms_list = [hard0_zoom,hard1_zoom,hard2_zoom, hard3_zoom, hard4_zoom, hard5_zoom, hard6_zoom]
contours_list = [hard0_contours,hard1_contours,hard2_contours, hard3_contours, hard4_contours, hard5_contours, hard6_contours]
for i in range(len(zooms_list)):
    for j in range(len(zooms_list[i])):
        figure(figsize=(6,6))
        ax = subplot(1,1,1)
        ax.tick_params(direction='in',labelleft=False, labelbottom = False, labelright = False)
        plt.imshow(zooms_list[i][j])
        cv.imwrite("../zoomed/trudne/trudne{}_{}_zoom.jpg".format(i,j), zooms_list[i][j])

NameError: name 'io' is not defined

In [3]:
areas = [[500, 1000],
         [1000,500,1970,1000],
         [900,900,900,1000],
         ]
ns = [[1,1],
      [1,1,0.5,1],
      [1,0.5,1,0.5], 
      ]
x = []
for i in range(20):
    x.append(i)
sides = [[[8,9],[8,9]],
         [x,x,x,x],
         [[8,9,10],[7,8,9,10],[8,9,10],[7,8,9,10]],
         ]
v = [[3,1],
     [1,1.5,1,1],
     [1,1,1.5,2],
     ]
for i in range(len(zooms_list)):
    for j in range(len(zooms_list[i])):
        kosteczki_hard(i,j, areas[i][j], ns[i][j], sides[i][j],v[i][j])

NameError: name 'zooms_list' is not defined

In [ ]:
def policzone_hard(img, indi, base_img_coords, x):
    font = cv.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (base_img_coords[1]-150, base_img_coords[0])
    fontScale = 8
    fontColor = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    lineType = 30
    output = str(indi)
    img = np.array(img)
    cv.putText(img, output, bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    #io.imshow(img)
    img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    cv.imwrite("../count/trudne/trudne{}_number.jpg".format(x), img)

def kosteczki_hard(x,y,are,n, sides, v):

    img = cv.imread('../zoomed/trudne/trudne{}_{}_zoom.jpg'.format(x,y))
    
    if y == 0:
        original = io.imread('../res/trudne{}.jpg'.format(x))
    else:
        try:
            original = io.imread("../count/trudne/trudne{}_number.jpg".format(x))
        except:
            original = io.imread('../res/trudne{}.jpg'.format(x))
    base_img_coords = contours_list[x][y]

    image = adjust_gamma(img, v)
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blur = cv.medianBlur(gray, 5)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv.filter2D(blur, -1, sharpen_kernel)

    #thresh = cv.adaptiveThreshold(image,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    thresh = cv.threshold(sharpen,130,255,cv.THRESH_BINARY_INV)[1]
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
    close = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)

    #plt.imshow(close)
    cnts = cv.findContours(close, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    fields_list = []
    approx_list = []
    x_list = []
    y_list = []
    for c in cnts:
        area = cv.contourArea(c)
        if area > 400:
            approx = cv.approxPolyDP(c, 0.02*cv.arcLength(c, True), True)
            approx_list.append(approx)
            x_list.append(approx.ravel()[0])
            y_list.append(approx.ravel()[1])
            fields_list.append(area)

    zip_list1 = zip(fields_list, approx_list)
    zip_list2 = zip(fields_list, x_list)
    zip_list3 = zip(fields_list, y_list)

    sorted_zip_list1 = sorted(zip_list1)
    sorted_zip_list2 = sorted(zip_list2)
    sorted_zip_list3 = sorted(zip_list3)

    approx_list = [element for _, element in sorted_zip_list1]
    x_list = [element for _, element in sorted_zip_list2]
    y_list = [element for _, element in sorted_zip_list3]
    fields_list = sorted(fields_list)
    try:
        mediana = statistics.median(fields_list)
    except:
        mediana = 1
    indi=0
    for i in range(0, len(fields_list)):
        if max(fields_list) > n*fields_list[i] and fields_list[i] > are:
            if (len(approx_list[i]) in sides) and abs(fields_list[i] - mediana) < 0.5*mediana:
                cv.drawContours(img, [approx_list[i]], 0, (255, 0, 0), 5)
                indi+=1
            
    if indi > 0:
        #figure(figsize=(6,6))
        #ax = subplot(1,1,1)
        #ax.tick_params(direction='in',labelleft=False, labelbottom = False, labelright = False)
        #wyświetlanie kostek
        #io.imshow(img)
        img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        cv.imwrite("../contours/trudne/trudne{}_{}_conturs.jpg".format(x,y), img)
        policzone_hard(original, indi, base_img_coords, x)
    
    #sprawdzanie pól i dlugosci approx
    #if x == 4:
    #    print(fields_list)
    #    for i in approx_list:
    #        print(len(i))
#interact(kosteczki, x=(0,9,1))